In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import numpy as np

In [3]:
# 加载数据集
data = load_breast_cancer()
X = data.data
y = data.target

# 数据标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 数据集划分
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# 将数据转换为 PyTorch 张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [4]:
# # 构建简单的二分类模型
# class SimpleNN(nn.Module):
#     def __init__(self, input_dim):
#         super(SimpleNN, self).__init__()
#         self.fc1 = nn.Linear(input_dim, 64)
#         self.fc2 = nn.Linear(64, 32)
#         self.fc3 = nn.Linear(32, 3)  # 输出3个类别
        
#     def forward(self, x):
#         x = torch.relu(self.fc1(x))
#         x = torch.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

In [43]:
# 构建具有SE Layer的二分类模型
class SELayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SELayer, self).__init__()
        # Squeeze阶段，使用全局平均池化
        self.fc1 = nn.Linear(channel, channel // reduction, bias=False)
        self.fc2 = nn.Linear(channel // reduction, channel, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Squeeze阶段: 全局平均池化
        b, c, _, _ = x.size()  # 获取batch size, 通道数，宽度和高度
        y = torch.mean(x, dim=[2, 3], keepdim=True)  # 全局平均池化
        y = y.view(b, c)  # 重塑为 [b, c]
        
        # Excitation阶段: 通过两层全连接网络和Sigmoid激活函数生成权重
        y = torch.relu(self.fc1(y))
        y = self.fc2(y)
        y = self.sigmoid(y).view(b, c, 1, 1)  # 重塑为 [b, c, 1, 1]

        # Scaling阶段: 重新调整权重到输入x
        return x * y.expand_as(x)

# 简单的神经网络模型
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)  # 输出2个类别
        
        # SE Layer, 假设输入特征经过fc2后为32个通道
        self.se = SELayer(32)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        
        # 在此处应用SE层
        x = self.se(x.unsqueeze(-1).unsqueeze(-1))  # 给x增加维度, [batch_size, 32, 1, 1]
        
        x = self.fc3(x.view(x.size(0), -1))  # 展平为 [batch_size, 32]
        return x

In [44]:
# 初始化模型, 定义损失函数和优化器
model = SimpleNN(X_train.shape[1])  # 输入维度为特征数量
criterion = nn.CrossEntropyLoss()  # 使用交叉熵损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam优化器

In [45]:
# 训练模型
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    
    # 前向传播
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    # 反向传播
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [10/20], Loss: 0.6286
Epoch [20/20], Loss: 0.5174


In [46]:
# 评估模型准确率
model.eval()  # 设置模型为评估模式
with torch.no_grad():
    # 预测
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    
    # 计算准确率
    accuracy = metrics.accuracy_score(y_test, predicted.numpy())
    print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 94.74%
